# 3. naive-submission
Kaggle score: 0.119

Abstrct:
- train table might have similar distribution of labels to submission.csv

Reference:
- https://www.kaggle.com/petrov/naive-submission-0-262

## Run name

In [1]:
import time

project_name = '3. naive-submission'
time_str = time.strftime("%Y%m%d-%H%M%S", time.localtime())
run_name = '%s_%s' % (project_name, time_str)
print(run_name)

time0 = time.time()

## Important params

In [2]:
top_n = 3
run_rows = 100*10000

print(top_n, run_rows)

3 1000000


## Import pkgs

In [3]:
import os
import random
import time
import re
from tqdm import tqdm
import multiprocessing
from tqdm import tqdm_notebook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, fbeta_score, jaccard_similarity_score

cpu_amount = multiprocessing.cpu_count()
random_num = np.random.randint(0, 9999)
random_num_str = '%04d' % random_num

print('cpu_amount: %s' % (cpu_amount - 1))
print('random_num: %s' % random_num_str)

C:\Users\study\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


cpu_amount: 3
random_num: 5300


## Project folders

In [4]:
cwd = os.getcwd()
feature_folder = os.path.join(cwd, 'feature')
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')
log_folder = os.path.join(cwd, 'log')

print(feature_folder)
print(input_folder)
print(output_folder)
print(model_folder)
print(log_folder)

train_human_labels_csv = os.path.join(input_folder, 'train_human_labels.csv')
stage_1_sample_submission_csv = os.path.join(input_folder, 'stage_1_sample_submission.csv')
print(train_human_labels_csv)
print(stage_1_sample_submission_csv)

D:\kaggle-Inclusive-Images\feature
D:\kaggle-Inclusive-Images\input
D:\kaggle-Inclusive-Images\output
D:\kaggle-Inclusive-Images\model
D:\kaggle-Inclusive-Images\log
D:\kaggle-Inclusive-Images\input\train_human_labels.csv
D:\kaggle-Inclusive-Images\input\stage_1_sample_submission.csv


In [5]:
# train_human_labels_df = pd.read_csv(train_human_labels_csv, names=['id', 'labels'], index_col=['id'])
train_human_labels_df = pd.read_csv(train_human_labels_csv, index_col=['ImageID'], nrows=run_rows)
display(train_human_labels_df.shape, train_human_labels_df.head())

(1000000, 3)

,Source,LabelName,Confidence
ImageID,,,
000002b66c9c498e,crowdsource-verification,/m/01kcnl,1
000002b66c9c498e,verification,/m/012mj,1
000002b66c9c498e,verification,/m/012yh1,1
000002b66c9c498e,verification,/m/014sv8,1
000002b66c9c498e,verification,/m/016c68,1


In [6]:
# calculate top_n most popular labels
predicted = ' '.join(
    train_human_labels_df['LabelName']
#     .str
#     .split()
    .apply(pd.Series)
    .stack()
    .value_counts()
    .head(top_n)
    .index
    .tolist()
)

print(f'{top_n} most popular labels are: {predicted}')

3 most popular labels are: /m/01g317 /m/01bqvp /m/09j2d


In [7]:
submission = pd.read_csv(stage_1_sample_submission_csv)

In [8]:
# use most popular labels as a prediction unless the correct labels are provided
submission['labels'] = predicted

display(submission.shape, submission.head())

(32580, 2)

,image_id,labels
0,2b2b327132556c767a736b3d,/m/01g317 /m/01bqvp /m/09j2d
1,2b2b394755692f303963553d,/m/01g317 /m/01bqvp /m/09j2d
2,2b2b42584e6d445937444d3d,/m/01g317 /m/01bqvp /m/09j2d
3,2b2b44744e57674270616f3d,/m/01g317 /m/01bqvp /m/09j2d
4,2b2b4b425a592b683059493d,/m/01g317 /m/01bqvp /m/09j2d


In [9]:
submission_csv_file = os.path.join(output_folder, '%s.csv' % run_name)
print(submission_csv_file)
submission.to_csv(submission_csv_file, index=False)

D:\kaggle-Inclusive-Images\output\3. naive-submission_20181016-203939.csv


In [10]:
print(run_name)
print('Time elapsed: %.2f s' % (time.time() - time0))

3. naive-submission_20181016-203939
Time elapsed: 160.57 s
